# Khan Academy Take Home Assignment (Program Analysis)
## Notebook By: Kevin Womack

### Introduction
The goal of this exercise is to understand the effect of program intervention on learner usage in the EdTech platform across various districts. In order to do this, I will need to design a visualization (likely with multiple views) that will allow us to examine usage change over time, both at an individual district level and by cross-sections of program and/or district characteristics.

But before beginning the process of designing and constructing these dashboards, I will use the `pandas` Python library to examine the cleanliness and key features of the dataset.

In [6]:
import pandas as pd

# Examine the shape (nrows, ncolumns) of the dataset
data = pd.read_csv('./data/program_analysis_data.csv')
data.shape

(5612, 10)

In [4]:
# Examine the top of the dataset
data.head()

,reporting_date,academic_term_name,partner_id,program_started,program_model,commitment_level,grade_levels,rostered_learners,active_learners,very_active_learners
0,2023-10-30,2023 - 2024 SY,JegVb+e1Bx2778GmzbS5sqSzN9gzjXto3x6Iby8KYPU=,False,NaN,NaN,NaN,270,231,20
1,2023-10-29,2023 - 2024 SY,Y2JPPwlGkiVUo3TXr4pBGFq/NpGn3btcuckvOW8nr2s=,False,NaN,NaN,NaN,5458,3102,14
2,2023-09-28,2023 - 2024 SY,rTvBTTUAKUjLmLGRxxq9Fm7c3DN5ZJtysHJ+pOoZFwY=,False,NaN,NaN,NaN,1338,1281,0
3,2023-09-09,2023 - 2024 SY,B++Z1VlCM2PfcXMo8HREN1XglSB2P981bhosh3YLrow=,False,NaN,NaN,NaN,2476,1645,3
4,2023-09-24,2023 - 2024 SY,ztctexxU7U2a70cB9CGC09kOwXpcmoQNNP98WXa0fpU=,False,NaN,NaN,NaN,15672,3988,22


In [3]:
# Examine the bottom of the dataset
data.tail()

,reporting_date,academic_term_name,partner_id,program_started,program_model,commitment_level,grade_levels,rostered_learners,active_learners,very_active_learners
5607,2024-11-22,2024 - 2025 SY,W88GuLNMt4ZhTe6tbQ6Oq037hFoq2YAbFFmFuAunENA=,True,Model B,Low,9-12,2293,1630,12
5608,2024-10-19,2024 - 2025 SY,VWe7Y86Kh7kTY28xVHpOWbPSVZNGgKwt4476xULNHn0=,True,Model A,High,9-12,963,791,4
5609,2024-09-29,2024 - 2025 SY,VWe7Y86Kh7kTY28xVHpOWbPSVZNGgKwt4476xULNHn0=,True,Model A,High,9-12,963,760,2
5610,2024-09-18,2024 - 2025 SY,VWe7Y86Kh7kTY28xVHpOWbPSVZNGgKwt4476xULNHn0=,True,Model A,High,9-12,963,745,2
5611,2024-10-28,2024 - 2025 SY,0MIw7EhCoutq79+6b722/hjxx1cpvUbfzAT6XClKVsg=,True,Model A,High,9-12,910,669,1


### Assumptions, Observations, and Validations

On loading the dataset, the first and foremost thing to confirm is whether or not it is in a "development-ready" state. In order to do this, I will use code to confirm some assumptions about the each column that were either given by the instruction prompt or assumed on initial inspection.

In [12]:
data.dtypes

reporting_date          object
academic_term_name      object
partner_id              object
program_started           bool
program_model           object
commitment_level        object
grade_levels            object
rostered_learners        int64
active_learners          int64
very_active_learners     int64
dtype: object

#### *Categorical Columns*

Continuing with the dataset inspection, one can use the `unique` method to validate the members of each piece of categorical data. Some data issues to look for could include:

- Nulls/Missing Data

- Dirty/Unprocessed Data (ex: misspellings, capitalization issues, etc.)

- Nonsensical data (ex: incorrect semester, etc.)

In [14]:
data['academic_term_name'].unique()

array(['2023 - 2024 SY', '2024 - 2025 SY'], dtype=object)

In [16]:
data['program_started'].unique()

array([False,  True])

In [20]:
data['partner_id'].unique()

array(['JegVb+e1Bx2778GmzbS5sqSzN9gzjXto3x6Iby8KYPU=',
       'Y2JPPwlGkiVUo3TXr4pBGFq/NpGn3btcuckvOW8nr2s=',
       'rTvBTTUAKUjLmLGRxxq9Fm7c3DN5ZJtysHJ+pOoZFwY=',
       'B++Z1VlCM2PfcXMo8HREN1XglSB2P981bhosh3YLrow=',
       'ztctexxU7U2a70cB9CGC09kOwXpcmoQNNP98WXa0fpU=',
       '+LVQIP7PJWOu/kztbrDnndnikPLVz0Ocb5lBpmyvgAs=',
       '0PfNnaafD6NBM5qr3kPV8LPsOzFJpiKtf/IX6hBK/fE=',
       'VWe7Y86Kh7kTY28xVHpOWbPSVZNGgKwt4476xULNHn0=',
       'Z3VI3AL16Qbk7ySD7gS0LXICxFlQ27kJw2wFwLG0i4k=',
       'sguiQ/DTm0tcUmmhX5WHXJcWNVLTkRatUJvkhXZz3bU=',
       'W88GuLNMt4ZhTe6tbQ6Oq037hFoq2YAbFFmFuAunENA=',
       'Etk0Z04DdqemXm9QSvrh4oTXwtuKAAlOVofU/IgxomA=',
       'OGlwbiCKI4Xt9RJOAwUvtrnaWIf8ySu4NWicAgto76s=',
       'gjQopZDPCM9u8OPQGq7ZVgU8yHz2l0WK37VXBCh2syg=',
       'LmttyVOgstljvJ8jOZWkvfgu9XAUykHpVovgfk0oiMk=',
       'odixc7Ugk2G0F5I0eX5Oc5tLR7qGR+al8u4oECdidAI=',
       'Vt7NoQZaPOqa6L8bmn/31XcOTIHwXAFAWD8QNzM2RXo=',
       'Orotcgqq0bXAgkfxkgwkW55k0abSlmNcEpsirZyllok=',
       'Lf

On review, the categorical columns above seem to have no issues, meaning that there is a value for every row and each value seems to make sense with respect to the nature of the data and the exercise. However, the categorical columns shown below seem to contain `NaN` or `Null` values.

In [17]:
data['program_model'].unique()

array([nan, 'Model B', 'Model A'], dtype=object)

In [18]:
data['commitment_level'].unique()

array([nan, 'Low', 'High'], dtype=object)

In [19]:
data['grade_levels'].unique()

array([nan, '11-12', '2-6', '3-12', '3-5', '3-8', '5-8', '6-11', '6-12',
       '8-11', '9-12'], dtype=object)

To begin to understand the reason for this, I returned to an observation that I made when using the `head` and `tail` methods earlier. It appears that certain pieces of data are simply unavailable or purposefully missing before start of the program.

In [27]:
# Isolate rows before program start
program_not_started = data[data['program_started'] == False]
program_not_started.shape

(2806, 10)

In [28]:
# Check for any missing data
program_not_started.isna().any()

reporting_date          False
academic_term_name      False
partner_id              False
program_started         False
program_model            True
commitment_level         True
grade_levels             True
rostered_learners       False
active_learners         False
very_active_learners    False
dtype: bool

In [22]:
# Isolate rows after program start
program_started = data[data['program_started'] == True]
program_started.shape

(2806, 10)

In [26]:
# Check for any missing data
program_started.isna().any()

reporting_date          False
academic_term_name      False
partner_id              False
program_started         False
program_model           False
commitment_level        False
grade_levels            False
rostered_learners       False
active_learners         False
very_active_learners    False
dtype: bool

For `program_model` this makes sense, because how can there be a model when there is no program? Though, for `grade_levels` and `commitment_level` the missing data is less understandable; and herein lies my first assumption:

**Assumption One: For the sake of visualization, we will assume that both `grade_levels` and `commitment_level` are consistent from one semester year to the next.**

This assumption allows us to maintain the notion that changes in learner usage can more surely be attributed to the interventions of the EdTech programs. Lastly, I wanted to confirm that the intervention of each program is isolated to only one semester year.

In [29]:
# Program has started by 2023-2024 SY
program_not_started['academic_term_name'].unique()

array(['2023 - 2024 SY'], dtype=object)

In [30]:
# Program has started by 2024-2025 SY
program_started['academic_term_name'].unique()

array(['2024 - 2025 SY'], dtype=object)

#### *Numerical Columns*

The validation of these columns is a bit more straightforward. Given that the measures of `rostered_learners`, `active_learners`, and `very_active_learners` are headcount totals and we've already confirmed that these columns don't have missing data, we simply need to check for nonsensical data, which would include:

- Negative numbers
- Fractions
- Outliers

For the sake of simplicity and brevity, we will include an exploration of the first of these.

In [35]:
# Check that each column only consists of natural numbers
data[['rostered_learners', 'active_learners', 'very_active_learners']].lt(0).any()

rostered_learners       False
active_learners         False
very_active_learners    False
dtype: bool

With the confirmation that each numerical column only consists of natural numbers, we proceed with examining the fluctuations in these measures over time to assess the effectiveness of each intervention program. However, if the daily changes in each of these columns can be attributed to other reasons (ex. mid-semester opt outs), the conclusions we draw from the data may not be as sure. With this in mind, there's one more assumption for this exercise that I'd like to submit:

**Assumption Two: Each daily change in learner headcount columns can be explained by the progression/regression of a learner from one category to another.**

This means that (within each district) the `rostered_learners`, `active_learners`, and `very_active_learners` daily counts should total the same number within a given semester year.

### Dashboard Design and Development

With the data validated and assumptions registered, the design of the dashboard could begin. As mentioned earlier, it was clear right away that delivering a complete product would require two views with distinctly different purposes and audiences.

The first view would allow both internal and external users to view usage growth in a given district by total changes in each learner measure over the course of the semester year. Additionally, lines for both pre- and post-program usage growth are plotted against each other so that the intervention effectiveness can be seen more clearly.

![District-Level View](https://github.com/kwomackcodes/Khan-Academy-Write-Up/blob/main/images/District%20Level%20View.jpg)

The second view is intended primarily for internal stakewholders who are interested in examining program effectiveness across district and program characteristics. The parameters in this view were designed to give specialized analysts with highly customizable views of intervention success.

![Program-Level View](./images/Program Level View.jpg)